# Unsupervised Learning of Depth and Ego-Motion from Video

# Approach
同时训练[Single-view Depth CNN]()和[Camera Pose Estimation CNN]()两个网络

用连续的3帧(中间帧被看作Target View, 其余帧被看作Source View),

## Single-view Depth CNN

使用[DispNet](#References), 基本上就是Encoder-Decoder + Skip Connections + Multi-scale Side Predictions. 除了输出层, 所有的卷积层都使用了ReLU激活函数, 而输出层使用了$1/(α∗sigmoid(x)+β)$ with $α = 10$ and $β = 0.1$将预测结果(Depth)限制在TODO: 求范围.
![](imgs/cnn1.png)


- 经过测试, 发现multi-views和single-view的性能差不多. 为了更好地利用多帧的信息, 应该加强对光流的限制
- 值得一提的是, 这篇论文通过类似Warping Loss的方法, 第一次将无监督学习应用到单目视觉的Depth信息恢复上来

## Camera Pose Estimation CNN
Pose Net的输入为Target View, Source Views在Channel通道的Concat后的结果, 输出为Poses. 网络由7个stride=2的卷积以及组成
应用Global Average Pooling来. 输出层不使用非线性函数, 其余层使用ReLU

![](imgs/cnn2.png)

Explanability Mask
![](imgs/mask.png)

## Explainability Prediction Network
与Depth Network和Pose Network一起同步(jointly and simultaneously)训练

每一个Target-Source Pair都会输出一个逐像素Soft mask, 表示Network的可信度

对于mask, 并没有直接的label, 直接用上述Loss, 模型可能会学到让E一直为0, 整个loss=0, 完美收敛(误)

加入了regularization项, 来鼓励输出非零的mask

## Differentiable Depth Image-based Rendering
![](imgs/differentiable_image_warping.png)

之前光流预测的工作中有接触到, 光流没有groundtruth怎么办? 根据光流将两帧通过warping对齐来计算L2 Loss

这一部分是构成Loss至关重要的组件, 通过前两个模型的预测(Depth Map和Pose)来将Source View对齐到Target View, 类似Spatial Transformer Networks中的方法来warp图像

// 在立体视觉中, 坐标映射可以表示为

由Target View到Source View的坐标映射可以表示为
$$p_s ~ KT_{t\rightarrow s}D_t(p_t)K^{-1}p_t$$

其中K为Camera Intrinsics Matrix

# Training

在讲述网络结构的过程中我们已经引入了一些对loss的介绍, 整理如下:
- 最简单的用来训练Depth Net和Pose Net的公式

- 引入Explainability regularization:
    $$$$
- 为了平滑, 引入L1 Loss:


loss的最终公式为:
$$L_{final} = \Sigma_l L^l_{vs} + \lambda_sL^l_{smooth} + \lambda_e\Sigma_sL_{reg}(E^l_s)$$
其中l代表了不同的scale, s代表了Source Views, 两个$\lambda$代表了系数

$$L_{vs} = \Sigma_s\Sigma_p |I_t(p) - I_s(p)|$$
即Target View和Warped Source Views的像素差求和

## Datasets

假设场景是刚性的(帧之间的一切变化由相机运动引起)

克服局部梯度

同Motion Estimation一样, 上述的Learning Pipeline存在一个问题, 那就是梯度主要由Target View和Source View之间变化强烈的像素决定. 这有可能会阻碍训练. 通常用以下两种方法:

- 使用small bottleneck的Encoder-Decoder架构, 这限制了output
- 

# Limitation
过强的假设
- 场景是静态的(没有运动物体)
- Target View和Source View之间没有occlusion/disocclusion(咬合/无咬合)
- 物体表面是Lambertian反射(又称理想反射), 好像是光流的那个假设?

# Future Work
- 在Single-view Depth Estimation中, 经过对比发现结果跟相比略逊一筹, 引入Cycle Consistency Loss可能会提高性能

# Evaluation

# References
